### Self-Organizing Novelty Detector

In [2]:
import numpy as np

In [58]:
def SONDE(X, sigma, alpha=0.1, beta=0.05, threshold=1e-3, eps=1e-7):
    state_prev = -1
    state = -1
    H = None
    M = None
    centroids = np.empty(0)
    
    # Para cada elemento (linha)
    for xi in X :
        # Verifico se existem centroids
        if (centroids.size == 0) :
            # Se não existir, primeiro xi é adicionado nos centroids
            centroids = np.array(xi)
            # Cria matriz m de tamanho 1x1 com zeros
            M = np.zeros((centroids.size -1, centroids.size -1))
            print(M)
            # state = id do novo centroid
            state = centroids.size -2
            print('ç')
            print(state)

        # Se já existirem centroids
        else:
            # Calcula a distância euclidiana de cada centroid existente para xi
            D = np.apply_along_axis(lambda cluster, xi: np.sqrt(np.sum(np.power(cluster-xi, 2))), 0, centroids, xi)
            
            # Aplica a função de ativação gaussiana nas distâncias dos centroids
            act = np.array([np.exp(-np.power(D, 2)/(2*np.power(sigma, 2)))])
            idx = np.argsort(act)[0] # Pega a maior ativação do centroid
            # Verifica se a ativação do é maior que o threshold
            if (act[idx] > threshold):
                # Se ativação é maior, state = id do centroid
                state = idx
                # centroid é deslocado em alpha * direção do x_i
                centroids[idx, :] = (1-alpha)*centroids[idx, :] + alpha*x_i
            else:
                # Se ativação é menor que threshold, novo centroid é criado
                centroids = np.append(centroids, xi)
                # Mais uma linha e coluna é adicionado a M com valores zeros.
                M = np.pad(M, [(0,1), (0,1)], 'constant', constant_values=(0))
                print(M.shape)
                # state = id do centroid adicionado
                state = centroids.size - 2
                print(state)

        if state_prev != -1:
            
            print(state)
            print(M)
            M[state_prev, state-1] = M[state_prev, state-1] + beta
            M = np.apply_along_axis(lambda row, EPS : row/np.sum(row) + EPS, 1, M, eps)
        
        idx = M > 0

        H = np.append(H, -np.sum(M[idx] * np.log2(M[idx]) ))
        
        state_prev = state
    

    return (centroids, H)
C, H = SONDE(X, sigma=0.1)

[[0.]]
ç
0
(2, 2)
2
2
[[0. 0.]
 [0. 0.]]


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in true_divide
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in greater


ValueError: operands could not be broadcast together with shapes (4,) (2,) 

In [5]:
X = np.random.uniform(low=-10.0, high=10.0, size=2*100).reshape((100, 2))